<h1>JetCat P300-Pro Stock Cycle Analysis<h1>

Package Import

In [1]:
import math as m
import numpy as np
import pandas as pd


<h2>Metric Calculations<h2>

<h3>Metric Characteristics<h3>

In [2]:
pressure_ratio = 3.55
mdot_stock = 0.5
stock_max_fuel_consumption_m = .980 # L/min
stock_max_fuel_consumption_s = stock_max_fuel_consumption_m/60 # L/s
stock_idle_fuel_consumption_m = 0.179 # L/min
stock_idle_fuel_consumption_s = stock_idle_fuel_consumption_m/60 # L/s
engine_mass = 2.730 # kg
min_exhaust_gas_temp = 480+273.15 # K
max_exhaust_gas_temp = 750+273.15 # K
idle_rpm = 35000 # (1/min)
max_rpm = 105000 # (1/min)
idle_thrust = 14 #N
max_stock_thrust = 300 #N
max_reported_exhaust_gas_velocity = (2160 * (1/3600) * 1000) # m/s
exhaust_gas_power_output = (90 * 1000)
max_sfc = 0.157 # (kg/Nh)

# https://www.jetcat.de/en/productdetails/produkte/jetcat/produkte/Professionell/p300%20pro

<h3>Metric Dimensions<h3>

In [8]:
starter_to_nozz_exit_length = .381
entrance_to_nozz_exit_length = starter_to_nozz_exit_length-.080
casing_width = .132
casing_width_with_mounts = .188

# nozzle
turbine_exit_outer_diameter = .091
nozzle_length = .104

# Entrance - Estimated
entrance_ID = .120 #m
entrance_A = (m.pi*((entrance_ID/2)**2))



<h4>Ambient Conditions<h4>

In [6]:
Pa = 101325 # Pa
Ta = 298 # K
rho_a = 1.225 # kg/m^3



<h4>Inlet to Compressor<h4>

In [9]:
# Conservation of mass 
u2 = mdot_stock/(rho_a*entrance_A)
Po2 = Pa * pressure_ratio